In [1]:
#!pip install pymystem3
#!pip install -U pymorphy2-dicts-ru

In [2]:
import gensim
import wget
import sys
import logging
import re
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from erdeni_nlp import pymorphy2_lemmas
import zipfile
import pymorphy2
import json
import pandas as pd
pd.set_option('display.max_rows', None)
from IPython.display import display
import streamlit as st

2022-11-28 01:06:12,928 : INFO : NumExpr defaulting to 4 threads.


In [3]:
morph = pymorphy2.MorphAnalyzer()

2022-11-28 01:06:15,424 : INFO : Loading dictionaries from C:\ProgramData\Anaconda3\lib\site-packages\pymorphy2_dicts_ru\data
2022-11-28 01:06:15.424 INFO    pymorphy2.opencorpora_dict.wrapper: Loading dictionaries from C:\ProgramData\Anaconda3\lib\site-packages\pymorphy2_dicts_ru\data
2022-11-28 01:06:15,485 : INFO : format: 2.4, revision: 417127, updated: 2020-10-11T15:05:51.070345
2022-11-28 01:06:15.485 INFO    pymorphy2.opencorpora_dict.wrapper: format: 2.4, revision: 417127, updated: 2020-10-11T15:05:51.070345


In [25]:
model = gensim.models.KeyedVectors.load_word2vec_format("220/model.bin", binary=True)

2022-11-28 01:08:40,238 : INFO : loading projection weights from 220/model.bin
2022-11-28 01:08:40.238 INFO    gensim.models.keyedvectors: loading projection weights from 220/model.bin
2022-11-28 01:08:45,084 : INFO : KeyedVectors lifecycle event {'msg': 'loaded (249333, 300) matrix of type float32 from 220/model.bin', 'binary': True, 'encoding': 'utf8', 'datetime': '2022-11-28T01:08:45.084535', 'gensim': '4.2.0', 'python': '3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'load_word2vec_format'}
2022-11-28 01:08:45.084 INFO    gensim.utils: KeyedVectors lifecycle event {'msg': 'loaded (249333, 300) matrix of type float32 from 220/model.bin', 'binary': True, 'encoding': 'utf8', 'datetime': '2022-11-28T01:08:45.084535', 'gensim': '4.2.0', 'python': '3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'load_word2vec_format'}


In [26]:
# Загрузить словарь
with open('vocabulary.json', "r") as f:
    vocabulary = json.loads(f.read())
    
with open('stopwords.json', "r") as f:
    stopwords_drops = json.loads(f.read())

In [27]:
text = """
Консультант вопрос:  Добрый день, какую операцию хотели бы?
Клиент ответ: Оплатить жкх.
Консультант вопрос: Вы иметь карта нашего банка?
Клиент ответ: Да.
Консультант вопрос: мы с вами провести операцию на банкомат или личном кабинете, 
Клиент ответ: Банкомат.
Консультант ответ: Хорошо, пожалуйста вставляйте карту и вводить код.
"""

In [28]:
the_keep_stopwords = list(map(lambda x: x.split("_")[0], stopwords_drops.keys()))

In [29]:
output = pymorphy2_lemmas(morph, text=text, drop_stopword=True, keep_stopwords=the_keep_stopwords)
#len(output), output

In [30]:
result_videos = []
not_in_model = set()
dataframe_data = []

def add_to_result(filename, input_lemma, output_lemma, similarity, queue=result_videos, df_dict=dataframe_data):
    queue.append(filename)
    df_dict.append({
        'input': input_lemma.split("_")[0],
        'output': output_lemma.split("_")[0],
        'similarity': similarity,
    })


for lemma in output:
    
    stopword_filename = stopwords_drops.get(lemma)
    if stopword_filename:
        add_to_result(stopword_filename, lemma, lemma, "stopword")
        continue
    
    similarity_max = 0
    similarity_word = None
    for iterated_word in vocabulary.keys():
        if lemma == iterated_word:
            filename = vocabulary.get(lemma)
            add_to_result(filename, lemma, lemma, "found")
            similarity_word = None
            break

        if lemma in model and iterated_word in model:
            percent = model.similarity(lemma, iterated_word)
            if percent > similarity_max:
                similarity_max = percent
                similarity_word = iterated_word
        else:
            #if lemma not in model:
            #    not_in_model.add(lemma)
            if iterated_word not in model:
                not_in_model.add(iterated_word)
    
    if similarity_word:
        filename = vocabulary.get(similarity_word)
        add_to_result(filename, lemma, similarity_word, similarity_max)

In [31]:
len(not_in_model), not_in_model

(74,
 {'австралия_NOUN',
  'автогонки_NOUN',
  'аллилуйя_INTJ',
  'англия_NOUN',
  'армения_NOUN',
  'беретка_NOUN',
  'важно_ADV',
  'венесуэла_NOUN',
  'виновный_NOUN',
  'влюблённость_NOUN',
  'вменяемый_ADJ',
  'вьетнам_NOUN',
  'г_NOUN',
  'германия_NOUN',
  'голландия_NOUN',
  'гребля_NOUN',
  'грузия_NOUN',
  'дом-интернат_NOUN',
  'египет_NOUN',
  'жкх_NOUN',
  'з_X',
  'занятый_ADJ',
  'израиль_NOUN',
  'индия_NOUN',
  'й_X',
  'канада_NOUN',
  'кенгуру_NOUN',
  'ковёр_NOUN',
  'конёк_NOUN',
  'копьё_NOUN',
  'кошелёк_NOUN',
  'л_NOUN',
  'латвия_NOUN',
  'леска_NOUN',
  'м_NOUN',
  'н_NOUN',
  'нет_ADV',
  'носка_NOUN',
  'орёл_NOUN',
  'осёл_NOUN',
  'отдый_ADJ',
  'п_NOUN',
  'польша_NOUN',
  'портной_NOUN',
  'преждевременно_ADV',
  'пупок_NOUN',
  'р_NOUN',
  'рапира_NOUN',
  'ружьё_NOUN',
  'румыния_NOUN',
  'слепоглухонемой_NOUN',
  'слесарить_VERB',
  'сливка_NOUN',
  'сурдолгий_ADJ',
  'сурдопереводчик_NOUN',
  'тапочки_NOUN',
  'татария_NOUN',
  'телёнок_NOUN',
  'ф_

In [10]:
df = pd.DataFrame(dataframe_data)
display(df)

,input,output,similarity
0,консультант,консультант,found
1,вопрос,вопрос,found
2,добрый,добрый,found
3,день,день,found
4,какой,какой,found
5,операция,операция,found
6,хотеть,желать,0.841871
7,клиент,клиент,found
8,ответ,ответ,found
9,оплатить,платить,0.657103


In [11]:
df.to_excel("Similarity.xlsx")

In [12]:
df['output'].to_list()

['консультант',
 'вопрос',
 'добрый',
 'день',
 'какой',
 'операция',
 'желать',
 'клиент',
 'ответ',
 'платить',
 'жкх',
 'консультант',
 'вопрос',
 'вы',
 'иметь',
 'карта',
 'наш',
 'банк',
 'клиент',
 'ответ',
 'да',
 'консультант',
 'вопрос',
 'мы',
 'с',
 'вы',
 'проводить',
 'операция',
 'банкомат',
 'или',
 'личный',
 'кабинет',
 'клиент',
 'ответ',
 'банкомат',
 'консультант',
 'ответ',
 'хорошо',
 'пожалуйста',
 'вставить',
 'карта',
 'и',
 'вводить',
 'код']